# Google Brain Ventilator Pressure Prediction Feature Enginering

- このnotebookの目的
    - [Google Brain コンペティション](https://www.kaggle.com/c/ventilator-pressure-prediction)について、モデルを作成するために有用そうな特徴量を作成する
    - 作成した特徴量をデータセットとして保存しておく
    
- このnotebookで出力されたデータセット
    - https://www.kaggle.com/tetsuya777/google-brain-ventilator-pressure-dataset

## 1. 準備

### 1.1 インポート

In [ ]:
# 基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# # 機械学習ライブラリ
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()

# # light GBM
# ! pip install lightgbm
# import lightgbm as lgb

# Warningの無効化
import warnings
warnings.filterwarnings('ignore')

# データフレームcolumの全表示
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

# hdf5 ファイル保存用 python パッケージ
import h5py

### 1.2 データセットの読み込み

In [ ]:
# データの読み込み
train_df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test_df = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

# まとめてデータ加工するために一度train or testのラベルを付けてデータをひとまとめにする
train_df.insert(0, 'dataset', 'train')
test_df.insert(0, 'dataset', 'test')
all_df = pd.concat([train_df, test_df])

# データ確認
all_df.head(83)

## 2 特徴量の作成

### 前後のtime_step間隔データの追加...**time_interval**

存在しないデータについては0.0を代入する

In [ ]:
all_df['time_interval_bf'] = all_df['time_step']-all_df['time_step'].shift(1)
all_df['time_interval_bf'][(all_df['breath_id']!=all_df['breath_id'].shift(1))] = 0.0 # データがない場合は0.0を代入

all_df['time_interval_af'] = all_df['time_step']-all_df['time_step'].shift(-1)
all_df['time_interval_af'][(all_df['breath_id']!=all_df['breath_id'].shift(-1))] = 0.0 # データがない場合は0.0を代入

all_df.head(83)

### u_out[i]-u_out[i-1]データの追加...**u_out_diff**

In [ ]:
all_df['u_out_diff'] = (all_df['u_out']-all_df['u_out'].shift(1))
all_df['u_out_diff'][(all_df['breath_id']!=all_df['breath_id'].shift(1))] = 0.0 # データがない場合は0.0を代入
all_df.head(83)

### u_out_diff/time_interval_bfデータの追加...**u_out_slope**

In [ ]:
all_df['u_out_slope'] = all_df['u_out_diff'] / all_df['time_step']
all_df['u_out_slope'][(all_df['breath_id']!=all_df['breath_id'].shift(1))] = 0.0 # データがない場合は0.0を代入
all_df.head(83)

### u_in[i]-u_in[i-1]データの追加...**u_in_diff**

In [ ]:
all_df['u_in_diff'] = (all_df['u_in']-all_df['u_in'].shift(1))
all_df['u_in_diff'][(all_df['breath_id']!=all_df['breath_id'].shift(1))] = 0.0 # データがない場合は0.0を代入
all_df.head(83)

### u_in/time_intervalデータの追加...**u_in_slope**

In [ ]:
all_df['u_in_slope'] = all_df['u_in_diff'] / all_df['time_step']
all_df['u_in_slope'][(all_df['breath_id']!=all_df['breath_id'].shift(1))] = 0.0 # データがない場合は0.0を代入
all_df.head(83)

### 0~20サンプル前のu_outデータの追加...**u_out_bf_(num)**

In [ ]:
num_shift = 10

for i_shift in range(1, num_shift+1):
    all_df[f'u_out_bf_{i_shift}'] = all_df['u_out'].shift(i_shift)
    change_breath_id_flag = np.array((all_df['breath_id'] != all_df['breath_id'].shift(i_shift)))
    all_df[f'u_out_bf_{i_shift}'][change_breath_id_flag] = 0.0 # データがない場合は0.0を代入
    all_df[f'u_out_bf_{i_shift}'] = all_df[f'u_out_bf_{i_shift}'].astype('float32')

all_df.head(83)

### 0~10サンプル前までのu_out_diffデータの追加...**u_out_diff_bf_(num)**

In [ ]:
num_shift = 10

for i_shift in range(1, num_shift+1):
    all_df[f'u_out_diff_bf_{i_shift}'] = all_df['u_out_diff'].shift(i_shift)
    change_breath_id_flag = np.array((all_df['breath_id'] != all_df['breath_id'].shift(i_shift)))
    all_df[f'u_out_diff_bf_{i_shift}'][change_breath_id_flag] = 0.0 # データがない場合は0.0を代入
    all_df[f'u_out_diff_bf_{i_shift}'] = all_df[f'u_out_diff_bf_{i_shift}'].astype('float32')

all_df.head(83)

### 0~10サンプル前までのu_out_slopeデータの追加...**u_out_slope_bf_(num)**

In [ ]:
num_shift = 10

for i_shift in range(1, num_shift+1):
    all_df[f'u_out_slope_bf_{i_shift}'] = all_df['u_out_slope'].shift(i_shift)
    change_breath_id_flag = np.array((all_df['breath_id'] != all_df['breath_id'].shift(i_shift)))
    all_df[f'u_out_slope_bf_{i_shift}'][change_breath_id_flag] = 0.0 # データがない場合は0.0を代入
    all_df[f'u_out_slope_bf_{i_shift}'] = all_df[f'u_out_slope_bf_{i_shift}'].astype('float32')

all_df.head(83)

### 0~30サンプル前のu_inデータの追加...**u_in_bf_(num)**

In [ ]:
num_shift = 30

for i_shift in range(1, num_shift+1):
    all_df[f'u_in_bf_{i_shift}'] = all_df['u_in'].shift(i_shift)
    change_breath_id_flag = np.array((all_df['breath_id'] != all_df['breath_id'].shift(i_shift)))
    all_df[f'u_in_bf_{i_shift}'][change_breath_id_flag] = 0.0 # データがない場合は0.0を代入
    all_df[f'u_in_bf_{i_shift}'] = all_df[f'u_in_bf_{i_shift}'].astype('float32')

all_df.head(83)

### 0~30サンプル前までのu_in_diffデータの追加...**u_in_diff_bf_(num)**

In [ ]:
num_shift = 30

for i_shift in range(1, num_shift+1):
    all_df[f'u_in_diff_bf_{i_shift}'] = all_df['u_in_diff'].shift(i_shift)
    change_breath_id_flag = np.array((all_df['breath_id'] != all_df['breath_id'].shift(i_shift)))
    all_df[f'u_in_diff_bf_{i_shift}'][change_breath_id_flag] = 0.0 # データがない場合は0.0を代入
    all_df[f'u_in_diff_bf_{i_shift}'] = all_df[f'u_in_diff_bf_{i_shift}'].astype('float32')

all_df.head(83)

### 0~30サンプル前までのu_in_slopeデータの追加...**u_in_slope_bf_(num)**

In [ ]:
num_shift = 30

for i_shift in range(1, num_shift+1):
    all_df[f'u_in_slope_bf_{i_shift}'] = all_df['u_in_slope'].shift(i_shift)
    change_breath_id_flag = np.array((all_df['breath_id'] != all_df['breath_id'].shift(i_shift)))
    all_df[f'u_in_slope_bf_{i_shift}'][change_breath_id_flag] = 0.0 # データがない場合は0.0を代入
    all_df[f'u_in_slope_bf_{i_shift}'] = all_df[f'u_in_slope_bf_{i_shift}'].astype('float32')

all_df.head(83)

### 0~30サンプル前までのu_inの平均値データの追加...**u_in_ave_bf_(num)**

In [ ]:
num_shift_list = [1, 2, 3, 4, 5, 10, 20, 30]
for i_range in num_shift_list:
    all_df[f'u_in_ave_bf_{i_range}'] = np.average(all_df.loc[:, f'u_in_bf_1':f'u_in_bf_{i_range}'], axis=1)
    all_df[f'u_in_ave_bf_{i_range}'] = all_df[f'u_in_ave_bf_{i_range}'].astype('float32')
    
all_df.head(83)

### 0~30サンプル前までのu_inの分散値データの追加...**u_in_var_bf_(num)**

In [ ]:
num_shift_list = [1, 2, 3, 4, 5, 10, 20, 30]
for i_range in num_shift_list:
    all_df[f'u_in_var_bf_{i_range}'] = np.var(all_df.loc[:, f'u_in_bf_1':f'u_in_bf_{i_range}'], axis=1)
    all_df[f'u_in_var_bf_{i_range}'] = all_df[f'u_in_var_bf_{i_range}'].astype('float32')
    
all_df.head(83)

### 0~30サンプル前までのu_in_slopeの平均値の追加...**u_in_slope_ave_bf_(num)**

In [ ]:
num_shift_list = [1, 2, 3, 4, 5, 10, 20, 30]
for i_range in num_shift_list:
    all_df[f'u_in_slope_ave_bf_{i_range}'] = np.var(all_df.loc[:, f'u_in_slope_bf_1':f'u_in_slope_bf_{i_range}'], axis=1)
    all_df[f'u_in_slope_ave_bf_{i_range}'] = all_df[f'u_in_slope_ave_bf_{i_range}'].astype('float32')
    
all_df.head(83)

### 0~30サンプル前までのu_in_slopeの分散値の追加...**u_in_slope_var_bf_(num)**

In [ ]:
num_shift_list = [1, 2, 3, 4, 5, 10, 20, 30]
for i_range in num_shift_list:
    all_df[f'u_in_slope_var_bf_{i_range}'] = np.var(all_df.loc[:, f'u_in_slope_bf_1':f'u_in_slope_bf_{i_range}'], axis=1)
    all_df[f'u_in_slope_var_bf_{i_range}'] = all_df[f'u_in_slope_var_bf_{i_range}'].astype('float32')
    
all_df.head(83)

### 3. ファイルの出力

In [ ]:
all_df

### hdf5データへの保存

- メリット:
    - 必要なcolumだけを読み取ることが可能(少しコーディングが必要)
    - 逐次保存を行うため保存時のメモリ使用量が大きくならない
    - gzipによる圧縮が可能(データサイズを半分にすることが可能)
- デメリット:
    - 保存、読み込みに時間がかかる(ただし、読み込み時はcolumn指定が可能)
- 参考:
    - https://qiita.com/sage-git/items/1d17d4058eca6a6a4826

In [ ]:
%%time
with h5py.File('all_df.h5', "w") as file:
    # 辞書と同じで順番が保持されないため、順番を保持したcolumns配列を保存
    file.create_dataset('columns', data=all_df.columns, dtype=h5py.special_dtype(vlen=str), compression="gzip")
    
    # データの格納
    for column in all_df.columns:
        dt = all_df[column].dtype if all_df[column].dtype!="object" else h5py.special_dtype(vlen=str)
        file.create_dataset(column, data=all_df[column], dtype=dt, compression="gzip")

読み出し例

In [ ]:
%%time
with h5py.File('all_df.h5', 'r') as file_r:
    # columsを全表示
    print(list(file_r.keys()))

    # データの読み込み
    all_df = pd.DataFrame([])
    for column, data in file_r.items():
        all_df[column] = data

In [ ]:
all_df

### pickleデータへの保存

- メリット:
    - 読み込み、書き込みにかかる時間が少ない
    - 読み込みが簡単(pd.read_pickleでOK)
- デメリット:
    - 圧縮ができないためファイルサイズが大きくなる
    - 書き込み時にデータフレームのコピーが行われるため、メモリが保存するデータサイズの2倍必要

In [ ]:
%%time
all_df.to_pickle('all_df.pkl')

以上